# 第３回講義

- ジェネレータ
- デコレータ
- コルーチン（Coroutine）
- yield
- @xxx
- PEP

## PEPを読もう

- Python Enhancement Proposal

## ジェネレータ関数

- yield文を含む関数は「ジェネレータ」を返す（ジェネレータ関数）

- ジェネレータはnext()に対してyield文の結果を順番に返す

In [1]:
# 従来の関数
def square_fun(n):
    return n * n

In [2]:
val = square_fun(5)
val

25

In [3]:
# ジェネレータ関数
# 関数を実行するためのオブジェクトが返される
def square_gen(n):
    yield n * n

In [4]:
# ジェネレータ関数からジェネレータが返される
# 関数は実行されない
g = square_gen(5)
g

<generator object square_gen at 0x7f8b2ffaa350>

In [5]:
val = next(g) # __next__に対してyield文の結果を順番に返す
val

25

In [6]:
def square_gen_1(n):
    for i in range(n):
        yield i * i

In [7]:
for x in square_gen_1(5):
    print(x)

0
1
4
9
16


- ジェネレータは関数が呼び出された直後は処理を実行しない

In [8]:
def square_gen_2(n):
    print("started")
    for i in range(n):
        yield i * i
    print("ended")

In [9]:
g = square_gen_2(5)
print("created")
for x in g:
    print(x)

created
started
0
1
4
9
16
ended


In [10]:
def foo(n):
    if n in (0, 1):
        return [1]
    for item in range(n):
        yield item * 2

In [11]:
g = foo(2)
for x in g:
    print(x)

0
2


- サブルーチンはコルーチンの一種

## FizzBuzz

In [12]:
def fizzbuzz(n):
    for i in range(1, n + 1):
        if i % 3 == 0 and i % 5 == 0:
            print("FizzBuzz")
        elif i % 3 == 0:
            print("Fizz")
        elif i % 5 == 0:
            print("Buzz")
        else:
            print(i)

In [13]:
fizzbuzz(15)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


## ジェネレータ版FizzBuzz

In [14]:
def fizzbuzz_gen():
    i = 1
    while True:
        if i % 3 == 0 and i % 5 == 0:
            yield "FizzBuzz"
        elif i % 3 == 0:
            yield "Fizz"
        elif i % 5 == 0:
            yield "Buzz"
        else:
            str(i)
        i += 1

In [15]:
for s in fizzbuzz_gen():
    print(s)
    if s == "FizzBuzz":
        break

Fizz
Buzz
Fizz
Fizz
Buzz
Fizz
FizzBuzz


## yield from

- ```yield from <expr>```で他のジェネレータに処理を移譲できる

In [16]:
def traverse(node):
    for elem in node:
        if isinstance(elem, int):
            yield elem
        else:
            yield from traverse(elem) # 再帰的に引数が再度traverseの中に入る

In [17]:
node = (1, (2, 3), 4, 5, (6, ((7, 8), 9)), 10)
for i in traverse(node):
    print(i)

1
2
3
4
5
6
7
8
9
10


## ジェネレータ関数におけるreturn

- ジェネレータ関数で、returnの結果はStopIteration例外に渡される

In [18]:
def square_gen_3(n):
    for i in range(n):
        yield i * i
        return "<finished>"

```Python
it = square_gen_3(3)
while True:
    print(next(it))
```
```
---------------------------------------------------------------------------
StopIteration                             Traceback (most recent call last)
<ipython-input-20-bb44bfa83be3> in <module>
      1 it = square_gen_3(3)
      2 while True:
----> 3     print(next(it))

StopIteration: <finished>
```

## ジェネレータ式

In [19]:
# ジェネレータ式
# 内包表記よりメモリ効率が良い
(x**2 for x in range(10))

<generator object <genexpr> at 0x7f8b30101190>

In [20]:
sum((x**2 for x in range(10))) # (ジェレータ式を明示するカッコ)は省略できる

285

## ジェネレータのsend(), throw()

- ジェネレータに値と例外を送ることができる

In [21]:
def gen_square(initial_i):
    i = initial_i
    try:
        while True:
            i = yield i ** 2
    except Exception as e:
        return 'Received "{}"'.format(e)

In [22]:
g = gen_square(1)
g

<generator object gen_square at 0x7f8b30101a50>

In [23]:
print(next(g))
print(g.send(2))

1
4


```Python
g.throw(RuntimeError("Stop")) # 例外を渡して処理を止める
```
```
---------------------------------------------------------------------------
StopIteration                             Traceback (most recent call last)
<ipython-input-24-c44525700109> in <module>
----> 1 g.throw(RuntimeError("Stop")) # 例外を渡して処理を止める

StopIteration: Received "Stop"
```

## コルーチン

- 並列化の一種
- ひとつのスレッドに複数の実行コンテクストを持つ仕組み

## ジェネレータはコルーチン

In [24]:
def calc_average(): # 平均値を与え続けるジェネレータ
    count, total, ave = 0, 0.0, None
    while True:
        val = yield ave
        count += 1
        total += val
        ave = total / count

In [25]:
coro_averave = calc_average()
next(coro_averave)

In [26]:
for val in [6, 1, 2, 5, 8]:
    print(coro_averave.send(val))

6.0
3.5
3.0
3.5
4.4


## asyncioモジュールと「ネイティブコルーチン」

- ジェネレータベースコルーチンとネイティブコルーチンがある

## デコレータとは

- 関数を返す関数

In [27]:
def as_is(func):
    return func

def get_hello():
    return "Hello"

In [28]:
get_hello = as_is(get_hello)
print(get_hello())

Hello


In [29]:
@as_is
def get_hello():
    return "Hello"

In [30]:
print(get_hello())

Hello


## 戻り値を加工するジェネレータ

In [31]:
def strong(func):
    def wrap():
        return "<strong>" + func() + "</strong>"
    return wrap

In [32]:
@strong
def get_hello():
    return "Hello"

In [33]:
print(get_hello())

<strong>Hello</strong>


## 引数を受け取るデコレータ

In [34]:
def tag(name):
    def deco(func):
        def wrap():
            return ("<{}>{}</{}>".format(name, func(), name))
        return wrap
    return deco

In [35]:
@tag("italic")
def get_hello():
    return "Hello"

In [36]:
print(get_hello())

<italic>Hello</italic>


## 修飾対象の引数を考慮する

In [37]:
def upper(func):
    def wrap(*args, **kargs):
        return func(*args, **kargs).upper()
    return wrap

In [38]:
@upper
def get_hello():
    return "Hello"

In [39]:
@upper
def get_something(something):
    return something

In [40]:
print(get_hello())
print(get_something("Good Evening"))

HELLO
GOOD EVENING


## デコレータを理解するうえでのヒント

- ジェネレータ関数と異なり、デコレータはただの関数
- 後から取り外せない

In [41]:
def deco_debug(func):
    def wrap(*args, **kargs):
        print(f'DEBUG: {func.__name__} started')
        ret = func(*args, **kargs)
        print(f'DEBUG: {func.__name__} ended')
        return ret
    return wrap

def say_hello():
    print("Hello")

In [42]:
say_hello()

Hello


In [43]:
deco_debug(say_hello)() # say_helloの呼び出しでエラーはあるかどうかチェックできる

DEBUG: say_hello started
Hello
DEBUG: say_hello ended


## functools.wraps()

- 主要な情報をコピーできる

In [44]:
from functools import wraps
def strong(func):
    @wraps(func)
    def wrap():
        return "<strong>" + func() + "</strong>"
    return wrap

@strong
def get_hello():
    """Helloという文字列を得る"""
    return "Hello"

In [45]:
get_hello()

'<strong>Hello</strong>'

In [46]:
# @wrapsで名前を承継できる
get_hello.__name__

'get_hello'

- ジェネレータ